# Hidden Markov Models

## Quick Recap of Probability

* Probability: measure of the **likelihood** of an event

* $0\leq p(x) \leq 1$, 
    * $p(x) = 0$ indicates that the event is very unlikely to occur 
    * $p(x) = 1$ indicates that the event will most likely occur
    
* 
    
## Quick Recap: Graph Theory

* Graphs: A tuple $\mathcal{G}=(\mathbf{X}, \mathbf{E})$ consisting in a set of nodes $\mathbf{X} = \{X_1,\dots,X_N\}$ and a set of edges $\mathbf{E}$ connecting the nodes.

* Edges can be *directed* $(X_i \rightarrow X_j)$ or *undirected* $(X_i - X_j)$. If the nodes of a graph are directed, we call it a *directed Graph*, otherwise is an *undirected graph*
* Parent and Children nodes: In directed graphs, if $(X_i \rightarrow X_j)\in \mathbf{E}$ then $X_i$ is a parent of $X_j$ and $X_j$ is a child of $X_i$. 
    * $\mathbf{Pa}(X_i)$ is the set of parents of $X_i$
    * $\mathbf{Ch}(X_i)$ is the set of children of $X_i$
    
Let's consider this graph

<div>
<img src="img/graph_example.png" width="250"/>
</div>

* the parents of $X_3$ are $\mathbf{Pa}(X_3) = \{X_1, X_2\}$.
* Which are the parents of $X_4$?
* Which are the parents of $X_1$?


## Quick Recap: Probabilistic Graphical Models

* Probabilistic graphical models (PGMs) provide a way to visualize the structure of a probabilisitc model

* Easy and elegant way to represent conditional independence properties

* **Bayesian Networks**: Nodes in a graph represent *random variable* and edges specify conditional independence properties:

$$p(X_1, \dots, X_N) = \prod_{i=1}^{N} p(X_i \mid \mathbf{Pa}(X_i))$$


For the example above

$$p(X_1, X_2, X_3, X_4, X_5, X_6) = p(X_1) p(X_2) p(X_3\mid X_1, X_2) p(X_4 \mid X_3) p(X_5 \mid X_3) p(X_6 \mid X_3)$$

## Markov Models

* The simplest way of modeling a sequence of observations is to treat them as independent

<div>
<img src="img/independent_markov.png" width="250"/>
</div>

$$p(\mathbf{x}_1\dots, \mathbf{x}_N) = \prod_{n=1}^{N} p(\mathbf{x}_i)$$

but this is a poor assumption for inherently sequential data (like music!)

* Easy way to model sequential data: the conditional distribution of each variable is independent of all previous observations except for the most recent: *first-order Markov chain*

<div>
<img src="img/first_order_markov.png" width="250"/>
</div>

$$p(\mathbf{x}_1, \dots, \mathbf{x}_N) = p(\mathbf{x}_1)\prod_{n=2}^{N}p(\mathbf{x}_n \mid \mathbf{x}_{n-1})$$

## Hidden Markov Models

### Model Definition

A hidden Markov Model (HMM) is a *state space model* that is not limited by the Markov assumption to any order. To do this, for each observation $\mathbf{x}_n$, we have a corresponding *hidden* (latent) variable $\mathbf{z}_n$ that satisfies the conditional independence property that $\mathbf{z}_{n-1}$ and $\mathbf{z}_{n+1}$ are independent give $\mathbf{z}_n$.

<div>
<img src="img/hmm_example.png" width="250"/>
</div>

* Since an HMM is a Bayesian network, its joint probability distribution is given by
$$
p(\mathbf{x}_1, \dots, \mathbf{x}_{N-1}, \mathbf{z}_{1},\dots \mathbf{z}_{N-1})=p(\mathbf{z}_1) \left[ \prod_{n=2}^N p(\mathbf{z}_n \mid \mathbf{z}_{n-1})\right]\prod_{n=1}^N p(\mathbf{x}_n \mid \mathbf{z}_n)
$$

* In HMMs, the hidden variables $\mathbf{z}_n$ are discrete and are typically represented by multinomial variables. Therefore, it is convenient to use a 1-of-$K$ coding scheme. 

* The conditional distribution $p(\mathbf{z}_n \mid \mathbf{z}_{n -1})$ can be represented by a matrix $\mathbf{A}$, commonly referred to as *transition probabilities*.

$$A_{ij} = p(z_{ni} = 1 \mid  z_{n-1,j} = 1)$$

where $0\leq A_ij \leq 1$ and $\sum_j A_{ij} = 1$.

$$p(\mathbf{z}_n \mid \mathbf{z}_{n-1}) = \prod_{i=1}^K \prod_{j=1}^K A_{ij}^{z_{n-1,j} \cdot z_{n,i}}$$

* Since $\mathbf{z}_1$ has no parents, the marginal distribution $p(\mathbf{z}_1)$ is represented by a vector of probabilities $\mathbf{\pi}$ with elements $\pi_k = p(z_{1k} = 1)$

$$p(\mathbf{z}_1) = \prod_{k=1}^K \pi_k^{z_{1k}}$$

* The distribution of the observed variables (*emission probabilities*) is modeled by $p(\mathbf{x}_n \mid \mathbf{z}_n, \mathbf{\phi})$, where $\mathbf{\phi}$ is the set of parameters of this distribution.

$$ p(\mathbf{x}_n \mid \mathbf{z}_n) = \prod_{k=1}^K p(\mathbf{x}_n \mid  \mathbf{\phi}_k)^{z_{nk}} $$

* The set of parameters of the HMM is then $\mathbf{\theta} = \{\mathbf{\pi}, \mathbf{A}, \mathbf{\phi}\}$

### Example: Weather forcasting

Here is a classical exaple for HMMs!

As mentioned above, we can fully specify the HMM by defining 3 element
* an observation model that computes $p(\mathbf{x}_n \mid \mathbf{z}_n)$
* the transition probability matrix $\mathbf{A}$
* the initial probability vector $\mathbf{\pi}$

We are going to use the [`hiddenmarkov` package](https://github.com/neosatrapahereje/hiddenmarkov), which we prepared to help you implement HMMs. (You might need to install it in your environment via `pip install python-hiddenmarkov`)

Here is an example observation model, for the case that $\mathbf{x}_i$ can take $M$ different states, and that the probability $p(\mathbf{x}_n = m \mid \mathbf{z}_n = k) = c_{n,k}$ is constant.

In [11]:
obs = ("normal", "jacket", "umbrella")
observations = ("normal", "umbrella", "jacket", "umbrella", "normal")
states = ("Sunny", "Raining")
observation_probabilities = np.array([[0.5, 0.1],
                                      [0.4, 0.3],
                                      [0.1, 0.6]])
transition_probabilities = np.array([[0.7, 0.4],
                                     [0.3, 0.6]])

observation_model = CategoricalObservationModel(
    observation_probabilities, obs
)

init_distribution = np.array([0.6, 0.4])

best_sequence, log_likelihood = viterbi_algorithm(
    obs, init_distribution, 
    observation_model, transition_probabilities,
    state_space=np.array(states)
)

print(best_sequence)

['Sunny' 'Sunny' 'Raining']


In [12]:
np.exp(observation_model("normal"))

array([0.5, 0.1])

In [13]:
from hiddenmarkov import ConstantTransitionModel, HMM

class CategoricalStringObservationModel(ObservationModel):

    def __init__(
            self,
            observation_probabilities,
            observations=None,
            use_log_probabilities=True
    ):
        super().__init__(use_log_probabilities=use_log_probabilities)

        self.observation_probabilities = observation_probabilities

        if observations is not None:
            self.observations = list(observations)
        else:
            self.observations = [
                str(i) for i in range(len(observation_probabilities))
            ]

    @property
    def observation_probabilities(self):
        if self.use_log_probabilities:
            return self._log_obs_prob
        else:
            return self._obs_prob

    @observation_probabilities.setter
    def observation_probabilities(self, observation_probabilities):
        self._obs_prob = observation_probabilities
        self._log_obs_prob = np.log(self._obs_prob)

    def __call__(self, observation, *args, **kwargs):
        idx = self.observations.index(observation)
        return self.observation_probabilities[idx]

    
states = ("a","b")
observation_probabilities = np.array([[0.5, 0.1],
                                          [0.4, 0.3],
                                          [0.1, 0.6]])

transition_probabilities = np.array([[0.0, 1.0],
                                          [1.0, 0.0]])

observation_model = CategoricalStringObservationModel(observation_probabilities )

transition_model = ConstantTransitionModel(transition_probabilities)

hmm = HMM(observation_model, transition_model, state_space=states)

path, prob = hmm.find_best_sequence([str(k) for k in np.random.randint(3, size=(12))], log_probabilities=False)
print(path, prob)

['a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b' 'a' 'b'] 3.600000000000001e-08


/Users/carlos/miniconda3/envs/miws22/lib/python3.7/site-packages/hiddenmarkov/__init__.py:612: RuntimeWarning: divide by zero encountered in log
  else:


### Three Inference Problems with HMMs

1. Given $\mathbf{\theta}$ and a sequence of observations, find the most likely sequence of hidden varibles
    * Viterbi Algorithm
    
2. Given $\mathbf{\theta}$ and a sequence of observations, find the probability of the observed sequence
    * Forward algorithm (not covered here)
    
3. Given sequences of observations, learn the model parameters $\mathbf{\theta}$
    * Maximum likelihood Using Expectation-Maximization (also not covered here!)
    
For a more detailed (and formal) description, see the tutorial by [Pernkopf et al., 2013](https://www2.spsc.tugraz.at/www-archive/downloads/PGM.pdf).

### Viterbi Algorithm

* In many applications, the hidden variables have some meaningful interpretation:
    * In speech recognition: find the most probable sequence of phonemes
    * In bioinformatics: Aligning DNA/RNA sequences
    * In MIR: Music alignment, chord recognition, key identification...
    
* Formally, we would like to find

$$\hat{\mathbf{Z}} = \arg \max_\mathbf{Z} p(\mathbf{X}, \mathbf{Z})$$

where $\mathbf{X} = \{\mathbf{x}_1, \dots, \mathbf{x}_N\}$ and $\mathbf{Z} = \{\mathbf{z}_1,\dots, \mathbf{z}_N \}$

* Direct optimization of the joint distribution might not be feasible!

* Viterbi Algorithm: Use dynamic programming (like in DTW!)

* We define 
$$\omega(\mathbf{z}_n) = \max_{\mathbf{z}_1, \dots, \mathbf{z}_{n - 1}}\log p(\mathbf{x}_1,\dots, \mathbf{x}_{n-1}, \mathbf{z}_1,\dots, \mathbf{z}_{n-1})$$

(compare to the Dynamic Time Warping distance!)

#### Algorithm 

**Inputs**  
    
* Sequence of observations $\mathbf{X}=\{\mathbf{x}_1, \dots, \mathbf{x}_N\}$
* Input probabilities $\mathbf{\pi}$
* Transition Matrix $\mathbf{A}$
* Observation model (to compute $p(\mathbf{x}_{n} \mid \mathbf{z}_n)$)

**Initialization**

We want to compute

$$\omega(\mathbf{z}_1) =  \log p(\mathbf{x}_1 \mid \mathbf{z}_1) + \log p(\mathbf{z}_1)$$

* For $k \in [1, K]$
    $$ \omega_{1k} = p(\mathbf{x}_n \mid  \mathbf{\phi}_k) + \log \pi_k $$

**Viterbi interation**

We want to compute

$$\omega(\mathbf{z}_n) = \log p(\mathbf{x}_{n} \mid \mathbf{z}_{n}) + \max_{\mathbf{z}_n} \left\{\log p(\mathbf{z}_{n} \mid \mathbf{z}_{n-1}) + \omega(\mathbf{z}_{n - 1})  \right\}$$

* For $n\in[2, N]$
    * For $k \in [1, K]$
        $$\omega_{nk} = p(\mathbf{x}_n \mid  \mathbf{\phi}_k) + \max_i\{ A_{ik} + \omega_{n-1, i} \}$$

In [8]:
import numpy as np
from collections import defaultdict

In [9]:
def viterbi_algorithm(observations, init_distribution, 
                      observation_model, transition_probabilities,
                      state_space=None):
    """
    Find the most probable sequence of latent variables given
    a sequence of observations

    Parameters
    ----------
    observations: iterable
       An iterable containing observations. The type of each
       element depends on input types accepted by the
       `observation_model`
    init_distribution: np.array
        A 1D vector of length n_states defining the initial
        probabilities of each state
    observation_model : callable
        A method for computiong the observation (emission) probabilities.
    transition_probabilities: np.ndarray
        A (n_states, n_states) matrix where component
        [i, j] represents the probability of going to state j
        from state i.

    Returns
    -------
    path: np.ndarray
        The most probable sequence of latent variables
    likelihood: float
        The log-likelihood of the best sequence.
    """
    # Initialize matrix for holding the best sub-sequence
    # log-likelihood
    n_states = len(transition_probabilities)
    omega = np.zeros((len(observations), n_states))
    # Initialize dictionary for tracking the best paths
    path = defaultdict(lambda: list())
    
    # Initiate for i == 0
    obs_prob = observation_model(observations[0])
    omega[0, :] = obs_prob + init_distribution
    # omega[0, :] = 0

    # Initialize path
    for i in range(n_states):
        path[i].append(i)

    # Viterbi recursion
    for i, obs in enumerate(observations[1:], 1):
        obs_prob = observation_model(obs)
        for j in range(n_states):
            # prob, state = 0, 0
            prob, state = max(
                [(omega[i - 1, k] + transition_probabilities[k, j], k)
                 for k in range(n_states)], 
                key=lambda x: x[0]
                )
            omega[i, j] = obs_prob[j] + prob

            # keep track of the best state
            path[j].append(state)
    
    # Get index of the best state
    best_sequence_idx = omega[-1, :].argmax()
    # Get best path (backtracking!)
    best_sequence = np.array(path[best_sequence_idx][::-1], dtype=int)
    if state_space is not None:
        best_sequence = state_space[best_sequence]
    # likelihood of the path
    path_likelihood = omega[-1, best_sequence_idx]

    return best_sequence, path_likelihood